# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, AssemblyProtein, Membrane, Virus, \
                        Sugars, Vector2, Vector3, Quaternion
be = BioExplorer('localhost:5000')
core = be.core_api()

In [ ]:
status = be.reset()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
glycan_folder = pdb_folder + 'glycans/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

### Configuration

In [ ]:
# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

## Camera

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.2490159598361429, 0.278924132285937, -0.031108605002474303,  0.926943706419231],
    position=[106.94343252645696, 90.75589800549, 155.9692706943393],
    target=[-23.94500293334849, -18.433590036060252, -21.072285585803407]
)

## Scene description

### Coronavirus

In [ ]:
def add_virus(
    name, position, position_seed=1, rotation_seed=2, 
    rotation=Quaternion(), open_conformation_indices=list(), 
    morphing_step=0.0):
    
    core.set_application_parameters(image_stream_fps=0)
    closed_conformation_indices = list()
    for i in range(nb_protein_s):
        if i not in open_conformation_indices:
            closed_conformation_indices.append(i)

    virus_protein_s = Protein(
        sources=[
            pdb_folder + '6vyb.pdb',  # Open conformation
            pdb_folder + 'sars-cov-2-v1.pdb'  # Closed conformation
        ],
        load_hydrogen=protein_load_hydrogen, occurences=nb_protein_s, 
        assembly_params=[
            11.5,
            position_seed, 0.0,
            rotation_seed, 0.1,
            morphing_step
        ],
        cutoff_angle=0.999,
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0),
        instance_indices=[
            open_conformation_indices, 
            closed_conformation_indices
        ])

    virus_protein_m = Protein(
        sources=[pdb_folder + 'QHD43419a.pdb'], load_hydrogen=protein_load_hydrogen,
        occurences=nb_protein_m, 
        assembly_params=[
            2.5,
            1, 0.1,
            rotation_seed + 1, 0.3,
            morphing_step
        ],
        cutoff_angle=0.999,
        rotation=Quaternion(0.99, 0.0, 0.0, 0.135))

    virus_protein_e = Protein(
        sources=[pdb_folder + 'QHD43418a.pdb'], load_hydrogen=protein_load_hydrogen, 
        occurences=nb_protein_e, 
        assembly_params=[
            3.0,
            3, 0.1,
            rotation_seed + 2, 0.3,
            morphing_step
        ],
        cutoff_angle=0.9999, 
        rotation=Quaternion(0.705, 0.705, -0.04, -0.04))

    virus_membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=12000)

    rna_sequence = None
    clip_planes = list()
    if add_rna:
        clip_planes.append([0, 0, -1, 15])
        import math
        rna_sequence = RNASequence(
            source=rna_folder + 'sars-cov-2.rna',
            assembly_params=[
                11.0, 
                position_seed + 3, 0.0,
                rotation_seed + 4, 0.0,
                morphing_step
            ],
            t_range=Vector2(0, 30.5 * math.pi), 
            shape=be.RNA_SHAPE_TREFOIL_KNOT,
            shape_params=Vector3(1.51, 1.12, 1.93))

    coronavirus = Virus(
        name=name, protein_s=virus_protein_s, 
        protein_e=virus_protein_e,
        protein_m=virus_protein_m, membrane=virus_membrane,
        rna_sequence=rna_sequence,
        assembly_params=[
            45.0, 
            position_seed + 5, 0.025,
            rotation_seed + 6, 0.4,
            morphing_step
        ]
    )

    be.add_virus(
        virus=coronavirus, position=position, rotation=rotation,
        representation=protein_representation, 
        atom_radius_multiplier=protein_radius_multiplier,
        clipping_planes=clip_planes
    )

    if add_glycans:
        # High-mannose
        indices_closed = [61, 122, 234, 603, 709, 717, 801, 1074]
        indices_open = [61, 122, 234, 709, 717, 801, 1074]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=high_mannose_paths, 
            indices=indices_closed, representation=glycan_representation)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=high_mannose_paths,
            indices=indices_open, representation=glycan_representation)

        # Complex
        indices_closed = [17, 74, 149, 165, 282, 331,
                          343, 616, 657, 1098, 1134, 1158, 1173, 1194]
        indices_open = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=complex_paths, 
            indices=indices_closed,
            representation=glycan_representation)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=complex_paths, 
            indices=indices_open, representation=glycan_representation)

        # O-Glycans
        for index in [323, 325]:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_PROTEIN_S_CLOSED, 
                site_indices=[index], representation=glycan_representation)
            be.add_sugars(o_glycan)

        # High-mannose glycans on Protein M
        indices = [5]
        high_mannose_glycans = Sugars(
            rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=name + '_' + be.NAME_PROTEIN_M, source=high_mannose_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(high_mannose_glycans)


        # Complex glycans on Protein E
        indices = [48, 66]
        complex_glycans = Sugars(
            rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
            protein_name=name + '_' + be.NAME_PROTEIN_E, source=complex_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(complex_glycans)
        
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_models_visibility(True)
    core.set_application_parameters(image_stream_fps=20)    

In [ ]:
add_virus(
    name='Coronavirus 1', position=Vector3(), 
    open_conformation_indices=[1], 
    position_seed=5, rotation_seed=2
)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1
params.soft_shadows = 0.2
params.fog_start = 400
params.fog_thickness = 250
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Animation

In [ ]:
from braynsmediamaker import MovieMaker
from IPython.display import clear_output

mm = MovieMaker(core)
output_folder = '/tmp'

for frame in range(1, 20):
    clear_output()
    print('Frame %d' % frame)
    add_virus(
        name='Coronavirus 1', position=Vector3(), 
        open_conformation_indices=[1], 
        position_seed=5 * frame + 1, rotation_seed=frame + 2
    )
    if True:
        '''Snapshot'''
        mm.create_snapshot(
            size=[540, 540], samples_per_pixel=16,
            path=output_folder + '/%05d.png' % frame)